In [150]:
import geopandas
import json

# Define the initial CityJSON file
citymodel = "montreal.json"

# Define the roads dataset
roads_file = "roads_3d.gpkg"

output = "montreal_final.json"

In [148]:
with open(citymodel) as file:
    cm = json.load(file)

len(list(cm["CityObjects"]))

28018

In [26]:
roads = geopandas.read_file(roads_file)

len(roads)

761

In [51]:
from shapely.geometry import Polygon, MultiPolygon

def surface_to_polygon(surface, vertices):
    """Return a Polygon from a CityJSON surface."""
    
    outer_ring = map(lambda i: vertices[i], surface[0])
    
    inner_rings = [map(lambda i: vertices[i], ring) for ring in surface[1:]]
    
    return Polygon(outer_ring, inner_rings)

def geom_to_multipolygon(geom, vertices):
    """Return a MultiPolygon from a CityJSON geometry.
    
    Only MultiSurfaces supported."""
    
    surfaces = []
    
    if geom["type"] == "MultiSurface":
        for surface in geom["boundaries"]:
            surfaces.append(surface_to_polygon(surface, vertices))
    
    return MultiPolygon(surfaces)

In [52]:
cobj = cm["CityObjects"][next(iter(cm["CityObjects"]))]

geom = cobj["geometry"][0]

g = geom_to_multipolygon(geom, cm["vertices"])

In [71]:
intersecting_roads = roads[roads["geometry"].intersects(g)]

In [134]:
from shapely.geometry import MultiLineString

valid_g = g.buffer(0)

parts = intersecting_roads["geometry"].intersection(valid_g)

for idx, road in intersecting_roads.iterrows():
    for att in road
    print(road["geometry"])
    print(road["geometry"].intersection(valid_g))

ID_TRC: 1380879
DEB_GCH: 0
FIN_GCH: 0
ARR_GCH: Le Plateau-Mont-Royal
SENS_CIR: 0.0
CLASSE: 6.0
LIE_VOIE: None
TYP_VOIE: rue
DIR_VOIE: Est
NOM_VOIE: Sherbrooke
DEB_DRT: 0
FIN_DRT: 0
ARR_DRT: Ville-Marie
LIM_GCH: Montréal
LIM_DRT: Montréal
geometry: MULTILINESTRING Z ((299564.520287114 5042000 40.9263916015625, 299574.109 5042030.069 40.59217071533203))
ID_TRC: 4014907
DEB_GCH: 0
FIN_GCH: 0
ARR_GCH: Le Plateau-Mont-Royal
SENS_CIR: -1.0
CLASSE: 6.0
LIE_VOIE: None
TYP_VOIE: rue
DIR_VOIE: None
NOM_VOIE: Berri
DEB_DRT: 0
FIN_DRT: 0
ARR_DRT: Le Plateau-Mont-Royal
LIM_GCH: Montréal
LIM_DRT: Montréal
geometry: MULTILINESTRING Z ((299566.86224 5042007.125525 40.88735961914062, 299543.97 5042021.473 38.75387954711914))
ID_TRC: 4014911
DEB_GCH: 0
FIN_GCH: 0
ARR_GCH: Ville-Marie
SENS_CIR: -1.0
CLASSE: 6.0
LIE_VOIE: None
TYP_VOIE: rue
DIR_VOIE: None
NOM_VOIE: Berri
DEB_DRT: 0
FIN_DRT: 0
ARR_DRT: Ville-Marie
LIM_GCH: Montréal
LIM_DRT: Montréal
geometry: MULTILINESTRING Z ((299578.3355426121 5042000 3

In [109]:
def linestring_to_geom(line, vertices):
    """Returns the boundary indices and new vertices"""
    
    offset = len(vertices)
    
    indices = [offset + i for i in range(len(line.coords))]
    verts = [list(c) for c in line.coords]
    
    return (indices, verts)

In [149]:
for objid, cobj in cm["CityObjects"].items():
    assert len(cobj["geometry"]) == 1
    
    citygeom = cobj["geometry"][0]
    
    geom = geom_to_multipolygon(citygeom, cm["vertices"])
    # We need to make the geometry valid
    geom = geom.buffer(0)
    
    intersecting_roads = roads[roads["geometry"].intersects(geom)]
    parts = intersecting_roads["geometry"].intersection(geom)
    
    boundaries = []
    semantics = {
        "values": [],
        "surfaces": []
    }
    
    for idx, road in intersecting_roads.iterrows():
        lines = road["geometry"].intersection(geom)
        if lines.type == "LineString":
            lines = MultiLineString([lines])
            
        for line in lines.geoms:
            b, v = linestring_to_geom(line, cm["vertices"])

            boundaries.append(b)
            cm["vertices"].extend(v)

            semantics["values"].append(len(semantics["surfaces"]))

            atts = dict(road)
            del atts["geometry"]

            semantics["surfaces"].append(atts)
    
    new_geometry = {
        "type": "MultiLineString",
        "boundaries": boundaries,
        "semantics": semantics
    }
    
    cobj["geometry"].append(new_geometry)

In [151]:
with open(output, "w") as out:
    json.dump(cm, out)